In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [ ]:
import pandas as pd
import numpy as np
lst_embeddings = []
df = pd.read_csv(r'/content/gdrive/MyDrive/balanced_train.csv')

In [ ]:
!pip install sentence-transformers
from sentence_transformers import SentenceTransformer
sbert_model = SentenceTransformer('bert-large-nli-mean-tokens')

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 85 kB 4.9 MB/s 
     |████████████████████████████████| 5.3 MB 37.5 MB/s 
     |████████████████████████████████| 1.3 MB 57.9 MB/s 
     |████████████████████████████████| 163 kB 74.8 MB/s 
     |████████████████████████████████| 7.6 MB 65.7 MB/s 
  Created wheel for sentence-transformers: filename=sentence_transformers-2.2.2-py3-none-any.whl size=125938 sha256=175a6b9c82e3c15170f5ee6c30af60f5aa8b484798833e230cc40f31c49c94e0
  Stored in directory: /root/.cache/pip/wheels/bf/06/fb/d59c1e5bd1dac7f6cf61ec0036cc3a10ab8fecaa6b2c3d3ee9
Successfully built sentence-transformers


Downloading:   0%|          | 0.00/391 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/191 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/3.96k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/2.00 [00:00<?, ?B/s]

...

Downloading:   0%|          | 0.00/112 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/466k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/401 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/232k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/229 [00:00<?, ?B/s]

In [ ]:
for idx in range(len(df['comment_text'])):
  if idx%3000==0:
    if idx==0:
       print(idx)
    else:
      print(idx, sentences_embeddings.shape)
  sentences_embeddings = sbert_model.encode(df['comment_text'][idx])
  lst_embeddings.append(sentences_embeddings)

0
3000 (1024,)
6000 (1024,)
9000 (1024,)
12000 (1024,)
15000 (1024,)
18000 (1024,)
21000 (1024,)
24000 (1024,)
27000 (1024,)
30000 (1024,)
33000 (1024,)


In [ ]:
df['vectors'] = lst_embeddings

In [ ]:
df.head()

,Unnamed: 0,id,comment_text,toxic,severe_toxic,obscene,threat,insult,identity_hate,Total Bias,vectors
0,6,0002bcb3da6cb337,COCKSUCKER BEFORE YOU PISS AROUND ON MY WORK,1,1,1,0,1,0,1,"[0.4711978, -0.34672344, 0.49185523, 0.0863054..."
1,12,0005c987bdfc9d4b,Hey... what is it..\n@ | talk .\nWhat is it......,1,0,0,0,0,0,1,"[0.04809183, -0.5674671, 0.21441089, 0.9148397..."
2,16,0007e25b2121310b,"Bye! \n\nDon't look, come or think of comming ...",1,0,0,0,0,0,1,"[-0.30192322, -0.5427696, 0.3625813, 0.9099709..."
3,42,001810bf8c45bf5f,You are gay or antisemmitian? \n\nArchangel WH...,1,0,1,0,1,1,1,"[0.96631056, -0.58349526, 0.21575046, 0.487635..."
4,43,00190820581d90ce,"FUCK YOUR FILTHY MOTHER IN THE ASS, DRY!",1,0,1,0,1,0,1,"[-0.741502, -1.0081497, 0.5087988, 0.7181236, ..."


In [ ]:
c = 0
lst_comment = []
lst_vector = []
lst_total_bias = []
df1 = pd.DataFrame()
for ind, rows in df.iterrows():
  if rows['Total Bias'] == 0 and c<13500:
    lst_comment.append(rows['comment_text'])
    lst_total_bias.append(0)
    lst_vector.append(rows['vectors'])
    c+=1

  elif rows['Total Bias'] == 1:
    lst_comment.append(rows['comment_text'])
    lst_total_bias.append(1)
    lst_vector.append(rows['vectors'])
  else:
    continue

df1['comment_text'] = lst_comment
df1['Total Bias'] = lst_total_bias
df1['vectors'] = lst_vector
type(df1['vectors'][1])

numpy.ndarray

In [ ]:
df1['Total Bias'].value_counts()

1    16225
0    13500
Name: Total Bias, dtype: int64

In [ ]:
df1.columns

Index(['comment_text', 'Total Bias', 'vectors'], dtype='object')

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import TensorDataset, DataLoader
from torch.utils.data.sampler import SubsetRandomSampler

In [ ]:
X = df1['vectors']
y = df1['Total Bias']
lst = []
for i in X:
  lst.append(i)
#print(type(lst[2]))

from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(lst, list(y), test_size=0.10, random_state=0)

x_train = np.array(x_train)
y_train = np.array(y_train)
x_test = np.array(x_test)
y_test = np.array(y_test)
# import pdb; pdb.set_trace();


pytorch_nn_model = nn.Sequential(
    nn.Linear(x_train.shape[1], 16),
    nn.ReLU(),
    nn.Linear(16, 16),
    nn.ReLU(),
    nn.Dropout(0.4),
    nn.Linear(16, 32),
    nn.ReLU(),
    nn.Dropout(0.3),
    nn.Linear(32, 1),
    nn.Sigmoid(),

)

tensor_x_train = torch.Tensor(x_train) # transform to torch tensor
tensor_y_train = torch.Tensor(y_train)
tensor_x_test = torch.Tensor(x_test) # transform to torch tensor
tensor_y_test = torch.Tensor(y_test)


tensor_y_train = tensor_y_train.type(torch.LongTensor)
tensor_y_test = tensor_y_test.type(torch.LongTensor)

train_dataset = TensorDataset(tensor_x_train,tensor_y_train)
test_dataset = TensorDataset(tensor_x_test,tensor_y_test)


train_loader = torch.utils.data.DataLoader(train_dataset)
test_loader = torch.utils.data.DataLoader(test_dataset)


PYDEV DEBUGGER WARNING:
sys.settrace() should not be used when the debugger is being used.
This may cause the debugger to stop working correctly.
If this is needed, please check: 
http://pydev.blogspot.com/2007/06/why-cant-pydev-debugger-work-with.html
to see how to restore the debug tracing back correctly.
Call Location:
  File "/usr/lib/python3.7/bdb.py", line 332, in set_trace
    sys.settrace(self.trace_dispatch)



--Return--
> <ipython-input-18-122491143910>(15)<module>()->None
-> import pdb; pdb.set_trace();
(Pdb) c



PYDEV DEBUGGER WARNING:
sys.settrace() should not be used when the debugger is being used.
This may cause the debugger to stop working correctly.
If this is needed, please check: 
http://pydev.blogspot.com/2007/06/why-cant-pydev-debugger-work-with.html
to see how to restore the debug tracing back correctly.
Call Location:
  File "/usr/lib/python3.7/bdb.py", line 343, in set_continue
    sys.settrace(None)



In [ ]:
pytorch_nn_model

Sequential(
  (0): Linear(in_features=1024, out_features=16, bias=True)
  (1): ReLU()
  (2): Linear(in_features=16, out_features=16, bias=True)
  (3): ReLU()
  (4): Dropout(p=0.4, inplace=False)
  (5): Linear(in_features=16, out_features=32, bias=True)
  (6): ReLU()
  (7): Dropout(p=0.3, inplace=False)
  (8): Linear(in_features=32, out_features=1, bias=True)
  (9): Sigmoid()
)

In [ ]:
import torch.optim as optim

criterion = nn.BCELoss()
optimizer = optim.Adam(pytorch_nn_model.parameters(), lr=0.001)

number_epochs = 20


def checksum(model):
    s = 0.0
    for param in model.parameters():
        s += torch.sum(param)
    return s

for epoch in range(number_epochs):
  trainLoss = 0
  trainAcc = 0
  samples = 0
  # Here, we use enumerate(training_loader) instead of
  # iter(training_loader) so that we can track the batch
  # index and do some intra-epoch reporting

  for i, data in enumerate(train_loader):



      # Every data instance is an input + label pair
      inputs, labels = data
      labels = labels.view(-1,1)
      # Zero your gradients for every batch!
      optimizer.zero_grad()

      # Make predictions for this batch
      outputs = pytorch_nn_model(inputs.float())

      # Compute the loss and its gradients
      loss = criterion(outputs, labels.float())
      loss.backward()

      # Adjust learning weights
      optimizer.step()

      trainLoss += loss.item() * labels.size(0)
      trainAcc += (torch.round(outputs) == labels.float()).sum().item()
      samples += labels.size(0)



    # display model progress on the current training batch
  trainTemplate = "epoch: {} train loss: {:.3f} train accuracy: {:.3f}"
  print(trainTemplate.format(epoch + 1, (trainLoss / samples),
      (trainAcc / samples)))




epoch: 1 train loss: 0.308 train accuracy: 0.876
epoch: 2 train loss: 0.269 train accuracy: 0.894
epoch: 3 train loss: 0.259 train accuracy: 0.899
epoch: 4 train loss: 0.257 train accuracy: 0.903
epoch: 5 train loss: 0.247 train accuracy: 0.904
epoch: 6 train loss: 0.240 train accuracy: 0.906
epoch: 7 train loss: 0.240 train accuracy: 0.906
epoch: 8 train loss: 0.242 train accuracy: 0.908
epoch: 9 train loss: 0.233 train accuracy: 0.909
epoch: 10 train loss: 0.231 train accuracy: 0.909
epoch: 11 train loss: 0.230 train accuracy: 0.911
epoch: 12 train loss: 0.230 train accuracy: 0.908
epoch: 13 train loss: 0.227 train accuracy: 0.911
epoch: 14 train loss: 0.225 train accuracy: 0.910
epoch: 15 train loss: 0.220 train accuracy: 0.911
epoch: 16 train loss: 0.228 train accuracy: 0.915
epoch: 17 train loss: 0.217 train accuracy: 0.914
epoch: 18 train loss: 0.227 train accuracy: 0.910
epoch: 19 train loss: 0.217 train accuracy: 0.914
epoch: 20 train loss: 0.217 train accuracy: 0.914


In [ ]:
trainLoss, trainAcc, samples  = 0, 0, 0

  
for i, data in enumerate(test_loader):



      # Every data instance is an input + label pair
      inputs, labels = data
      labels = labels.view(-1,1)
      # Zero your gradients for every batch!
      optimizer.zero_grad()

      # Make predictions for this batch
      outputs = pytorch_nn_model(inputs.float())

      # Compute the loss and its gradients
      loss = criterion(outputs, labels.float())
      loss.backward()

      # Adjust learning weights
      optimizer.step()

      trainLoss += loss.item() * labels.size(0)
      trainAcc += (torch.round(outputs) == labels.float()).sum().item()
      samples += labels.size(0)


In [ ]:
print("TESTING ACCURACY IS ---- ",(trainAcc/samples)*100,"%")

TESTING ACCURACY IS ----  91.41372607655502 %


In [ ]:



torch.save(pytorch_nn_model.state_dict(), '/content/gdrive/MyDrive/PPLM Code/Bert_large_pytorch_weights.pth')
torch.save(pytorch_nn_model, '/content/gdrive/MyDrive/PPLM Code/BERT_large_full_pytorch_model.pth')


In [ ]:
model = torch.load('/content/gdrive/MyDrive/PPLM Code/BERT_large_full_pytorch_model.pth')
model.eval() 

Sequential(
  (0): Linear(in_features=1024, out_features=16, bias=True)
  (1): ReLU()
  (2): Linear(in_features=16, out_features=16, bias=True)
  (3): ReLU()
  (4): Dropout(p=0.4, inplace=False)
  (5): Linear(in_features=16, out_features=32, bias=True)
  (6): ReLU()
  (7): Dropout(p=0.3, inplace=False)
  (8): Linear(in_features=32, out_features=1, bias=True)
  (9): Sigmoid()
)